In [1]:
# training_ssd.ipynb
import sys
from pathlib import Path
import yaml
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection import ssd300_vgg16
from torchvision.models.detection.ssd import SSDClassificationHead
from utils import get_transform, collate_fn
from traffic_dataset import TrafficDataset
from trainer import train_and_validate, prepare_save_dir


In [2]:
project_root = Path().resolve()
sys.path.append(str(project_root))

def load_config(path):
    with open(path, 'r') as file:
        return yaml.safe_load(file)

cfg = load_config(project_root / 'cfg' / 'ssd_day.yaml')
data_cfg = load_config(project_root / 'dataset' / 'dataset.yaml')

In [3]:
train_dataset_day = TrafficDataset(
    csv_file='dataset/annotations_train_balanced_day.csv',
    root_dir=Path("dataset/day/train"),
    transforms=get_transform(train=True, cfg=cfg)
)
val_dataset_day = TrafficDataset(
    csv_file='dataset/annotations_test_balanced_day.csv',
    root_dir=Path("dataset/day/test"),
    transforms=get_transform(train=False, cfg=cfg)
)

train_loader_day = DataLoader(train_dataset_day, batch_size=cfg['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=8)
val_loader_day = DataLoader(val_dataset_day, batch_size=cfg['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=8)

In [4]:
save_dir_day = prepare_save_dir(project_root / "runs_ssd" / "ssd_day_balanced", cfg)

# Model setup
model_day = ssd300_vgg16(pretrained=True)
in_channels = [m.in_channels for m in model_day.head.classification_head.module_list]
num_anchors = model_day.anchor_generator.num_anchors_per_location()
model_day.head.classification_head = SSDClassificationHead(in_channels, num_anchors, data_cfg['nc'])

/workspace/traffic-detection/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/workspace/traffic-detection/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth
100%|██████████| 136M/136M [00:01<00:00, 88.2MB/s] 


In [5]:
print(torch.cuda.is_available())
print(torch.version.cuda)
torch.cuda.empty_cache()

True
12.1


In [6]:
train_and_validate(model_day, cfg, data_cfg, train_loader_day, val_loader_day, save_dir_day)

[Epoch 1] Validation: 100%|██████████| 116/116 [00:53<00:00,  2.19it/s]


loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
Loading and preparing results...


[Epoch 2] Validation: 100%|██████████| 116/116 [00:54<00:00,  2.14it/s]


loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
Loading and preparing results...


[Epoch 3] Validation: 100%|██████████| 116/116 [00:56<00:00,  2.05it/s]


loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
Loading and preparing results...


[Epoch 4/50] Training:  84%|████████▍ | 169/200 [02:13<00:24,  1.27it/s, loss=24.9]


KeyboardInterrupt: 

In [ ]:
# train_dataset_night = TrafficDataset(
#     csv_file='dataset/annotations_train_night.csv',
#     root_dir=Path("dataset/night/train"),
#     transforms=get_transform(train=True, cfg=cfg)
# )
# val_dataset_night = TrafficDataset(
#     csv_file='dataset/annotations_test_night.csv',
#     root_dir=Path("dataset/night/test"),
#     transforms=get_transform(train=False, cfg=cfg)
# )

In [ ]:
# train_loader_night = DataLoader(train_dataset_night, batch_size=cfg['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=8)
# val_loader_night = DataLoader(val_dataset_night, batch_size=cfg['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=8)

In [ ]:
# save_dir_night = prepare_save_dir(project_root / "runs_ssd" / "ssd_night", cfg)

# model_night = ssd300_vgg16(pretrained=True)
# in_channels = [m.in_channels for m in model_night.head.classification_head.module_list]
# num_anchors = model_night.anchor_generator.num_anchors_per_location()
# model_night.head.classification_head = SSDClassificationHead(in_channels, num_anchors, data_cfg['nc'])

In [ ]:
# train_and_validate(model_night, cfg, data_cfg, train_loader_night, val_loader_night, save_dir_night)